# Factor Investing and Fama-French model

**Copyright &copy; 2020 Ondrej Martinsky, All rights reserved**

[www.github.com/omartinsky/FamaFrench](https://github.com/omartinsky/FamaFrench)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as wx
from ipywidgets import interact
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from pandas.plotting import register_matplotlib_converters
from IPython.core.display import display, HTML
import famafrench
register_matplotlib_converters()

/var/folders/fj/4vv2tyt94_l44bxlschbrx5h0000gn/T/ipykernel_90513/3355595522.py:9: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
/Users/marmarti/Documents/GITProjects/FamaFrench/MMFamaFrench/famafrench.py:54: FutureWarning: DataFrame.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  p = p.replace(-99.99, math.nan).replace(999) / 100


/Users/marmarti/Documents/GITProjects/FamaFrench/MMFamaFrench
Univariate SMB.csv
Module 'io' is installed.
/Users/marmarti/Documents/GITProjects/FamaFrench/MMFamaFrench
.DS_Store
Error decoding file content.
Module 'io' is installed.
Error decoding file content.


/Users/marmarti/Documents/GITProjects/FamaFrench/MMFamaFrench/famafrench.py:54: FutureWarning: DataFrame.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  p = p.replace(-99.99, math.nan).replace(999) / 100


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 3131: invalid start byte

In [ ]:
def create_ma_radios(value):
    return wx.RadioButtons(value=value, options=famafrench.lookup_ma.keys(), description='Roll Avg')

# Fama French

$ r = R_f + b_m \cdot \text{MER} + b_s \cdot \text{SMB} + b_v \cdot \text{HML} + b_p \cdot \text{RMW} + b_i \cdot \text{CMA} $

$r$ is expected portfolio return \
$b_m$, $b_s$, $b_v$, $b_p$, $b_i$ are factor sensitivities

# Factors Analysis
$\text{MER}$ is excess return of a broad market over the risk-free rate ($R_m - R_f$) (**market factor**) \
$\text{SMB}$ is the average return on the *small* stock portfolios minus *big* stock portfolios (**size factor**) \
$\text{HML}$ is the average return on *high* book value portfolios minus *low* book value (**value factor**)  \
$\text{RMW}$ is the average return on *robust* minus *weak* operating profitability portfolios (**profitability factor**) \
$\text{CMA}$ Conservative minus aggresive investments, proxied by the y/y growth in company asset value (**investment factor**)

Definitions: https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/Data_Library/f-f_5_factors_2x3.html

In [ ]:
display(HTML('<b>Factors time series</b>'), famafrench.df_factors)
display(HTML('<b>Factors Statistics</b>'), famafrench.df_factors.describe().T)
display(HTML('<b>Factors Correlation Matrix</b>'), famafrench.df_factors.corr())

In [ ]:
display(HTML("<b>Distribution Analysis</b>"))
    
wx_ff=wx.Dropdown(options=famafrench.lookup_factors.keys(), description='Factor')
wx_ma=create_ma_radios('1D')
wx_out1=wx.interactive_output(famafrench.plot_factor_hists, {'ff':wx_ff, 'ma':wx_ma})
wx_out2=wx.interactive_output(famafrench.plot_factor_timeseries, {'ff':wx_ff, 'ma':wx_ma})
display(wx.HBox([wx_ff, wx_ma]), wx_out1, wx_out2)

# Portfolio Analysis

In [ ]:
def create_factor_radios():
    return {
        'f_mer': wx.Checkbox(description='MER', indent=False, layout=chkbox_layout, value=True),
        'f_smb': wx.Checkbox(description='SMB', indent=False, layout=chkbox_layout, value=True),
        'f_hml': wx.Checkbox(description='HML', indent=False, layout=chkbox_layout, value=True),
        'f_rmw': wx.Checkbox(description='RMW', indent=False, layout=chkbox_layout),
        'f_cma': wx.Checkbox(description='CMA', indent=False, layout=chkbox_layout),
    }

def create_portfolio_dropdown():
    return wx.Dropdown(options=famafrench.portfolios.keys(), description='Portfolio', layout=wx.Layout(width='500px'))


chkbox_layout = wx.Layout(width='70px')
wx_pname = create_portfolio_dropdown()
wx_ma = create_ma_radios('3Y')
wx_factor_list = create_factor_radios()
wx_daterange=wx.SelectionRangeSlider(options=range(1960,2021), index=(0, len(range(1960,2021))-1), description='Range', continuous_update=False, layout=wx.Layout(width='500px'))
wx_out=wx.interactive_output(famafrench.fit_portfolio_returns, {'pname':wx_pname, **wx_factor_list , 'ma': wx_ma, 'daterange': wx_daterange})
wx_out.layout.height='700px'
wx_out_r2hist = wx.interactive_output(famafrench.draw_R2_hist, {'pname':wx_pname, 'daterange': wx_daterange})
wx_out_r2series = wx.interactive_output(famafrench.draw_R2_series, {'pname':wx_pname, **wx_factor_list, 'daterange': wx_daterange})
display(
    wx_pname, 
    wx.HBox([wx_daterange, *(wx_factor_list.values())]),
    wx.HBox([wx_out_r2hist, wx_out_r2series]),
    wx.HBox([wx_out, wx_ma]))